# PART 1 : Install Dependencies & Run Spark Session

In [1]:
#install pyspark
! pip install pyspark

from pyspark.sql.functions import when, col

In [2]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# PART 2: Clone & Explore dataset

In [3]:
#clone the diabetes dataset from the github repository
# if you get error saying "Project-4 directory already exists, move to next cell"
! git clone https://github.com/elluis1001/Project-4/

fatal: destination path 'Project-4' already exists and is not an empty directory.


In [4]:
#check if the dataset exists
! ls /content/Project-4/Luis/Dataset/

diabetes_prediction_dataset.csv  diabetes_test_dataset.csv


In [5]:
#create spark dataframe
df_diabetes_data = spark.read.csv("/content/Project-4/Luis/Dataset/diabetes_prediction_dataset.csv", header=True, inferSchema=True)

In [6]:
#display the dataframe
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|Female|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|Female|54.0|           0|            0|        No Info|27.32|        6.6|                 80|       0|
|  Male|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|Female|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|  Male|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|Female|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|Female|44.0|           0|            0|          never|19.31|  

In [7]:
#show amount of rows
df_diabetes_data.count()

100000

In [8]:
#print the schema
df_diabetes_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- diabetes: integer (nullable = true)



In [9]:
#count the total no. of diabetic and non-diabetic class (values of 1 indicating the presence of diabetes and 0 indicating the absence of diabetes)
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('diabetes').count().show()

(100000, 9)
+--------+-----+
|diabetes|count|
+--------+-----+
|       1| 8500|
|       0|91500|
+--------+-----+



In [10]:
#count the total no. of gender types
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(100000, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|58552|
| Other|   18|
|  Male|41430|
+------+-----+



In [11]:
#check to see if there are any empty values in the 'gender' column
df_diabetes_data[df_diabetes_data['gender'] == '']

DataFrame[gender: string, age: double, hypertension: int, heart_disease: int, smoking_history: string, bmi: double, HbA1c_level: double, blood_glucose_level: int, diabetes: int]

In [12]:
#get the summary statistics
df_diabetes_data.describe().show()

+-------+------+-----------------+------------------+------------------+---------------+-----------------+------------------+-------------------+-------------------+
|summary|gender|              age|      hypertension|     heart_disease|smoking_history|              bmi|       HbA1c_level|blood_glucose_level|           diabetes|
+-------+------+-----------------+------------------+------------------+---------------+-----------------+------------------+-------------------+-------------------+
|  count|100000|           100000|            100000|            100000|         100000|           100000|            100000|             100000|             100000|
|   mean|  NULL|41.88585600000013|           0.07485|           0.03942|           NULL|27.32076709999422|5.5275069999983275|          138.05806|              0.085|
| stddev|  NULL|22.51683987161704|0.2631504702289171|0.1945930169980986|           NULL|6.636783416648357|1.0706720918835468|  40.70813604870383|0.27888308976661896|
|   

# PART 3: Data Cleaning & Preparation

In [13]:
#check for null values
for col in df_diabetes_data.columns:
  print(col + ":", df_diabetes_data[df_diabetes_data[col].isNull()].count())

gender: 0
age: 0
hypertension: 0
heart_disease: 0
smoking_history: 0
bmi: 0
HbA1c_level: 0
blood_glucose_level: 0
diabetes: 0


In [14]:
#look for the unnecessary values present
def count_zeros():
  columns_list = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]
  for i in columns_list:
    print(i+":",df_diabetes_data[df_diabetes_data[i]==0].count())

In [15]:
count_zeros()

age: 0
bmi: 0
HbA1c_level: 0
blood_glucose_level: 0


In [16]:
#display the dataframe
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|Female|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|Female|54.0|           0|            0|        No Info|27.32|        6.6|                 80|       0|
|  Male|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|Female|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|  Male|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|Female|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|Female|44.0|           0|            0|          never|19.31|  

In [17]:
#drop the 'other' rows in the gender columns
string_to_remove = "Other"
df_diabetes_data = df_diabetes_data[df_diabetes_data['Gender'] != string_to_remove]

In [18]:
#count the total no. of gender types
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(99982, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|58552|
|  Male|41430|
+------+-----+



In [19]:
#count the total no. of smoker/non-smoker types
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('smoking_history').count().show()

(99982, 9)
+---------------+-----+
|smoking_history|count|
+---------------+-----+
|    not current| 6439|
|         former| 9352|
|        No Info|35810|
|        current| 9286|
|          never|35092|
|           ever| 4003|
+---------------+-----+



In [20]:
#drop the 'other' rows in the gender columns
string_to_remove_1= "No Info"
df_diabetes_data = df_diabetes_data[df_diabetes_data['smoking_history'] != string_to_remove_1]

In [21]:
#count the total no. of smoker/non-smoker types
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('smoking_history').count().show()

(64172, 9)
+---------------+-----+
|smoking_history|count|
+---------------+-----+
|    not current| 6439|
|         former| 9352|
|        current| 9286|
|          never|35092|
|           ever| 4003|
+---------------+-----+



In [22]:
#count the total no. of gender types
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(64172, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|38852|
|  Male|25320|
+------+-----+



In [23]:
#assign in the 'gender'column 'Female' = 0, and 'Male' = 1
from pyspark.sql.functions import when, col
df_diabetes_data = df_diabetes_data.withColumn("gender",
    when(col("gender") == "Female", 0).
    when(col("gender") == "Male", 1).
    otherwise(col("gender"))
)
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|     0|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|     1|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|     0|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|     1|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|     0|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|     0|44.0|           0|            0|          never|19.31|        6.5|                200|       1|
|     1|42.0|           0|            0|          never|33.64|  

In [24]:
#assign in the 'smoking_history': "never" = 0, "ever" = 1, "not current" = 2, "current" = 3, "former" = 4
df_diabetes_data = df_diabetes_data.withColumn("smoking_history",
    when(col("smoking_history") == "never", 0).
    when(col("smoking_history") == "ever", 1).
    when(col("smoking_history") == "not current", 2).
    when(col("smoking_history") == "current", 3).
    when(col("smoking_history") == "former", 4).
    otherwise(col("smoking_history"))
)
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|     0|80.0|           0|            1|              0|25.19|        6.6|                140|       0|
|     1|28.0|           0|            0|              0|27.32|        5.7|                158|       0|
|     0|36.0|           0|            0|              3|23.45|        5.0|                155|       0|
|     1|76.0|           1|            1|              3|20.14|        4.8|                155|       0|
|     0|20.0|           0|            0|              0|27.32|        6.6|                 85|       0|
|     0|44.0|           0|            0|              0|19.31|        6.5|                200|       1|
|     1|42.0|           0|            0|              0|33.64|  

# PART 4: Correlation Analysis & Feature Selection

In [25]:
# gender and smoking_history needs to be converted to float data type for model to work
df_diabetes_data = df_diabetes_data.withColumn("gender", col("gender").cast('float'))
df_diabetes_data = df_diabetes_data.withColumn("smoking_history", col("smoking_history").cast('float'))
df_diabetes_data.show()


+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|   0.0|80.0|           0|            1|            0.0|25.19|        6.6|                140|       0|
|   1.0|28.0|           0|            0|            0.0|27.32|        5.7|                158|       0|
|   0.0|36.0|           0|            0|            3.0|23.45|        5.0|                155|       0|
|   1.0|76.0|           1|            1|            3.0|20.14|        4.8|                155|       0|
|   0.0|20.0|           0|            0|            0.0|27.32|        6.6|                 85|       0|
|   0.0|44.0|           0|            0|            0.0|19.31|        6.5|                200|       1|
|   1.0|42.0|           0|            0|            0.0|33.64|  

In [26]:
#find the correlation among the set of input & output variables
for i in df_diabetes_data.columns:
  print("Correlation to outcome for {} is {}".format(i, df_diabetes_data.stat.corr("diabetes",i)))

Correlation to outcome for gender is 0.05699689368565596
Correlation to outcome for age is 0.26084962459224337
Correlation to outcome for hypertension is 0.19222574901207254
Correlation to outcome for heart_disease is 0.16961397731730365
Correlation to outcome for smoking_history is 0.06472564826560573
Correlation to outcome for bmi is 0.20442115545137657
Correlation to outcome for HbA1c_level is 0.43889709468177335
Correlation to outcome for blood_glucose_level is 0.449697968864106
Correlation to outcome for diabetes is 1.0


In [27]:
#feature selection
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['gender', 'age', 'hypertension', 'heart_disease',
                                         'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'], outputCol='features')
output_data = assembler.transform(df_diabetes_data)

In [28]:
#print the schema
output_data.printSchema()

root
 |-- gender: float (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: float (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
#display dataframe
output_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+--------------------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|            features|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+--------------------+
|   0.0|80.0|           0|            1|            0.0|25.19|        6.6|                140|       0|[0.0,80.0,0.0,1.0...|
|   1.0|28.0|           0|            0|            0.0|27.32|        5.7|                158|       0|[1.0,28.0,0.0,0.0...|
|   0.0|36.0|           0|            0|            3.0|23.45|        5.0|                155|       0|[0.0,36.0,0.0,0.0...|
|   1.0|76.0|           1|            1|            3.0|20.14|        4.8|                155|       0|[1.0,76.0,1.0,1.0...|
|   0.0|20.0|           0|            0|            0.0|27.32|        6.6|                 85|       0|(8,[1,5,6,7],[20....|


# PART 5: Split Dataset & Build the Model

In [30]:
#create final data
from pyspark.ml.classification import LogisticRegression

final_data = output_data.select('features','diabetes')

In [31]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- diabetes: integer (nullable = true)



In [32]:
#split the dataset ; build the model
train, test = final_data.randomSplit([0.7, 0.3])
models = LogisticRegression(labelCol= 'diabetes')
model = models.fit(train)

In [33]:
#summary of the model
summary = model.summary
summary.predictions.describe().show()

+-------+-------------------+------------------+
|summary|           diabetes|        prediction|
+-------+-------------------+------------------+
|  count|              44868|             44868|
|   mean|0.11036819113845056|0.0825978425603994|
| stddev| 0.3133516266524633|0.2752764571390363|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



# PART 6: Evaluate and Save the Model

In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.evaluate(test)

In [35]:
predictions.predictions.show(100)

+--------------------+--------+--------------------+--------------------+----------+
|            features|diabetes|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(8,[1,5,6,7],[0.4...|       0|[6.73298317644645...|[0.99881044196804...|       0.0|
|(8,[1,5,6,7],[0.4...|       0|[4.92667428830366...|[0.99280161587908...|       0.0|
|(8,[1,5,6,7],[0.8...|       0|[10.8965795842457...|[0.99998147886611...|       0.0|
|(8,[1,5,6,7],[1.0...|       0|[6.32328744905691...|[0.99820917680200...|       0.0|
|(8,[1,5,6,7],[1.0...|       0|[11.9291336362679...|[0.99999340461391...|       0.0|
|(8,[1,5,6,7],[1.3...|       0|[8.43037173799067...|[0.99978190719060...|       0.0|
|(8,[1,5,6,7],[1.3...|       0|[11.0352896938244...|[0.99998387767847...|       0.0|
|(8,[1,5,6,7],[1.5...|       0|[6.61259376465083...|[0.99865845883998...|       0.0|
|(8,[1,5,6,7],[1.6...|       0|[5.21786021822782...|[0.9946102943

In [36]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', labelCol='diabetes')
evaluator.evaluate(model.transform(test))

0.9548805640975272

In [37]:
# save model
model.save("model")

In [38]:
# load saved model back to the environment
from pyspark.ml.classification import LogisticRegressionModel

model = LogisticRegressionModel.load('model')

# PART 7: Prediction on New Data with the saved model


In [39]:
#create a new spark dataframe
test_df = spark.read.csv('/content/Project-4/Luis/Dataset/diabetes_test_dataset.csv', header=True, inferSchema=True)

In [40]:
#print the schema
test_df.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)



In [41]:
#create an additional feature merged column
test_data = assembler.transform(test_df)

In [42]:
#print the schema
test_data.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- features: vector (nullable = true)



In [43]:
#use model to make predictions
results = model.transform(test_data)
results.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [44]:
#display the predictions
results.select('features','rawPrediction','probability','prediction').show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[0.0,80.0,1.0,1.0...|[-0.1564843933134...|[0.46095853772191...|       1.0|
|[1.0,28.0,0.0,0.0...|[4.65830826051093...|[0.99060658225279...|       0.0|
|[0.0,36.0,0.0,0.0...|[6.60029094405418...|[0.99864187470865...|       0.0|
|[1.0,76.0,1.0,1.0...|[3.79800206745135...|[0.97807592747868...|       0.0|
|(8,[1,5,6,7],[20....|[5.68861253797044...|[0.99662712947532...|       0.0|
|(8,[1,5,6,7],[44....|[1.65750771129390...|[0.83990315918791...|       0.0|
|[1.0,42.0,0.0,0.0...|[5.94665930994294...|[0.99739225771866...|       0.0|
|(8,[1,5,6,7],[32....|[8.34281265525147...|[0.99976195497669...|       0.0|
+--------------------+--------------------+--------------------+----------+



# PART 8: Host a Flask App which will take subjects parameters and return probablity and prediction on being diabetic or not based on ML model above

In [149]:
# import dependencies to run Flask app and host it on publicly accessible colab URL
from flask import *
from google.colab import output
from google.colab.output import eval_js

In [150]:
# initialize Flask app
app=Flask(__name__, template_folder='/content/Project-4/Luis/U.I. for project/templates')

In [151]:
# Render Home page for Diabetes Prediction to take subject parameters as input
@app.route('/')
def home():
    return render_template('index.html')

In [152]:
# Dynamic API that will take parameters of subject through Web UI and leverage ML Model above to return probability & prediction for being diabetic
@app.route('/api/v1.0/predict/')
def predict(gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level):
    # create a tuple from input parameters and convert them to int or float as they are treated as string when passed from Web UI
    data = [(int(gender),int(age),int(hypertension),int(heart_disease),int(smoking_history),float(bmi),float(HbA1c_level),int(blood_glucose_level))]
    columns = ['gender','age','hypertension','heart_disease','smoking_history','bmi','HbA1c_level','blood_glucose_level']

    # Create spark dataframe for the input parameters
    test_df = spark.createDataFrame(data,columns)

    # invoke ML Model and capture model output in results
    test_data = assembler.transform(test_df)
    results = model.transform(test_data)

    # return the results after converting it to JSON
    return results.toJSON().first()

In [153]:
# Flask app when run gives local IP to access API. Since this is running on Cloud (Google Colab) and not on local notebook, local IP (127.0.0.1) will not be accessible.
# Below code will ask colab to give us publicly accessible URL

print(eval_js("google.colab.kernel.proxyPort(5000)"))
output.serve_kernel_port_as_window(5000)
if __name__ == '__main__':
    app.run(host='0.0.0.0',port=5000)

https://6yvrqczkyof-496ff2e9c6d22116-5000-colab.googleusercontent.com/


<IPython.core.display.Javascript object>

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:49:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:49:20] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:49:41] "GET /api/v1.0/predict/1/23/0/0/0/0/7/100 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:49:42] "GET /api/v1.0/predict/1/23/0/0/0/0/7/100 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:49:48] "GET /api/v1.0/predict/1/23/0/0/0/0/7/100 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:50:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:50:18] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Feb/2024 05:50:20] "GET /api/v1.0/predict/N